In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import io
from tensorflow.keras.models import load_model

# Path to the trained model (modify if needed)
MODEL_PATH = "/content/drive/MyDrive/best_efficientnetb0_model_1.h5"

# 1) Load the model without recompiling
model = load_model(MODEL_PATH, compile=False)

# 2) Print and save the model architecture summary
buf = io.StringIO()
model.summary(print_fn=lambda x: buf.write(x + "\n"))
summary_str = buf.getvalue()

print(summary_str)  # print the full summary
with open("/content/model_summary.txt", "w") as f:
    f.write(summary_str)

print("Model summary saved to /content/model_summary.txt")


Model: "sequential_4"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘
 Total params: 4,213,668 (16.07 MB)
 Trainable params:

In [3]:
# 1. Imports
import os, glob
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input

# 2. Paths and constants
IMG_DIR    = "/content/drive/MyDrive/feature"                     # directory of images to visualize
OUT_DIR    = "/content/drive/MyDrive/feature_cam_effnetb0"        # output directory
MODEL_PATH = "/content/drive/MyDrive/best_efficientnetb0_model_1.h5"  # trained EfficientNetB0 model
IMG_SIZE   = (224, 224)
os.makedirs(OUT_DIR, exist_ok=True)

# 3. Load model
base_model = load_model(MODEL_PATH, compile=False)

# 4. Backbone and last convolutional layer
backbone = base_model.get_layer("efficientnetb0")  # main feature extractor
LAST_CONV_NAME = "top_conv"                        # last convolutional layer
last_conv_layer = backbone.get_layer(LAST_CONV_NAME)

# Feature sub-model: input -> (last conv features, backbone output)
feat_model = Model(
    inputs=backbone.input,
    outputs=[last_conv_layer.output, backbone.output],
    name="effnetb0_feat_model"
)

# 5. Rebuild the full forward graph (backbone -> custom head)
inp = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3), name="gradcam_input")
conv_feat_tensor, bb_out = feat_model(inp)

x = bb_out
attach = False
for lyr in base_model.layers:
    if attach:
        x = lyr(x)
    if lyr.name == backbone.name:
        attach = True

func_model = Model(inputs=inp, outputs=x, name="effnetb0_functional_rebuilt")
grad_model = Model(inputs=func_model.input,
                   outputs=[conv_feat_tensor, func_model.output])

print(f"Grad-CAM model ready. Last convolutional layer: {LAST_CONV_NAME}")

# 6. Grad-CAM core utilities
def make_gradcam_heatmap(img_array, use_logit=False):
    """Compute Grad-CAM heatmap. Returns (heatmap, probability_of_AI)."""
    with tf.GradientTape() as tape:
        conv_out, preds = grad_model(img_array, training=False)
        p = preds[:, 0]  # Dense(1, sigmoid) -> probability of AI class
        if use_logit:
            eps = 1e-7
            p = tf.clip_by_value(p, eps, 1 - eps)
            target = tf.math.log(p / (1 - p))
        else:
            target = p
    grads = tape.gradient(target, conv_out)
    pooled = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_map = conv_out[0]
    heatmap = tf.reduce_sum(conv_map * pooled, axis=-1)
    heatmap = tf.nn.relu(heatmap)
    heatmap = heatmap / (tf.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy(), float(p.numpy()[0])

def overlay_heatmap(heatmap, bgr_img, alpha=0.45):
    """Overlay the heatmap on a BGR image."""
    h, w = bgr_img.shape[:2]
    heatmap = cv2.resize(heatmap, (w, h))
    heatmap_uint8 = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)
    return cv2.addWeighted(heatmap_color, alpha, bgr_img, 1 - alpha, 0)



Grad-CAM model ready. Last convolutional layer: top_conv


In [4]:
# 7. Batch processing: display and save
label_map = {0: "Human", 1: "AI"}
allow_ext = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
img_paths = sorted(glob.glob(os.path.join(IMG_DIR, "*.*")))

processed = 0
for p in img_paths:
    if os.path.splitext(p)[1].lower() not in allow_ext:
        continue

    bgr = cv2.imread(p)
    if bgr is None:
        print(f"Skipped: cannot read {p}")
        continue

    bgr_resized = cv2.resize(bgr, IMG_SIZE)

    ximg = image.img_to_array(cv2.cvtColor(bgr_resized, cv2.COLOR_BGR2RGB))
    ximg = np.expand_dims(ximg, axis=0)
    ximg = preprocess_input(ximg)  # EfficientNet preprocessing

    heatmap, prob_ai = make_gradcam_heatmap(ximg, use_logit=False)
    pred_cls = 1 if prob_ai >= 0.5 else 0

    cam_bgr = overlay_heatmap(heatmap, bgr_resized, alpha=0.45)

    # Prepare for display with matplotlib (expects RGB)
    rgb_orig = cv2.cvtColor(bgr_resized, cv2.COLOR_BGR2RGB)
    rgb_cam  = cv2.cvtColor(cam_bgr,     cv2.COLOR_BGR2RGB)

    # Plot: show and save
    plt.figure(figsize=(6.5, 3.2), dpi=200)
    plt.subplot(1, 2, 1); plt.imshow(rgb_orig); plt.axis("off"); plt.title("Original", fontsize=9)
    plt.subplot(1, 2, 2); plt.imshow(rgb_cam);  plt.axis("off")
    plt.title(f"Grad-CAM - Pred: {label_map[pred_cls]} (AI prob = {prob_ai:.2f})", fontsize=9)
    plt.tight_layout()

    out_name = os.path.splitext(os.path.basename(p))[0] + "_cam_effnetb0.jpg"
    out_path = os.path.join(OUT_DIR, out_name)
    plt.savefig(out_path, bbox_inches="tight")
    plt.show()
    plt.close()

    processed += 1
    print(f"Saved: {out_path}")

print(f"Done. Exported and displayed {processed} EfficientNetB0 Grad-CAM images to {OUT_DIR}.")


Output hidden; open in https://colab.research.google.com to view.